# Dynamic Programming with GridWorld

In [1]:
import numpy as np

## Intro

### Single-Row Gridworld

This is a minimal reinforcement learning environment designed to introduce fundamental concepts of grid-based agents and navigation.

### Grid Layout & Components

```
| S |   |   | G |
```

- **Total Cells**: 4
- **Start State (S)**: First cell (index 0)
- **Goal State (G)**: Last cell (index 3)

### Agent Actions

The agent can perform two basic actions:

1. **Left**: Move one cell to the left
   - If at the start state (S), no movement occurs
2. **Right**: Move one cell to the right
   - If at the goal state (G), no movement occurs

### State Transition Rules

#### Movement Constraints
- Cannot move beyond grid boundaries
- Attempting an invalid move keeps the agent in the current state

### Action Dynamics

#### Transition Probability
- All actions are deterministic (100% chance of expected outcome)

#### Reward Structure
- **Default Reward**: 0 for non-terminal state transitions
- **Goal Reward**: 1 when reaching the goal state

### State Representation

#### State Encoding
- **States**: 0 (S), 1, 2, 3 (G)
- **Terminal State**: State 3 (Goal state)

### Mathematical Representation

The environment can be formally described as:

```
action_i: {state_i: [(probability, next_state, reward, done)]}
```

Where:
- `probability`: Likelihood of state transition (1.0 in this deterministic environment)
- `next_state`: Resulting state after action
- `reward`: Value received for the transition
- `done`: Boolean indicating terminal state

### Detailed Transition Dynamics
- Left action from state 0: Remains at state 0
- Right action from state 3: Remains at state 3
- Other right/left actions move accordingly

### Learning Objective

The goal is to learn the optimal policy to reach the goal state with the maximum reward.

## Environment set up

In [2]:
class GridWorld:
    def __init__(self):
        self.n_states = 4  # We have 4 celles in the grid
        self.n_actions = 2  # We have 2 actions (left and right)
        # R = 0 for all transitions except terminal = 1
        self.P = { 
            0: {
                0: [
                    (1.0, 0, 0, False)
                ],  # from state 0, action 0 (left) leads to state 0 with reward 0. Agent bounced back and stayed in same position
                1: [
                    (1.0, 1, 0, False)
                ],  # from state 0, action 1 (right) leads to state 1 on the right with reward 0
            },
            1: {
                0: [
                    (1.0, 0, 0, False)
                ],  # from state 1, action 0 (left) leads to state 0 on the left with reward 0
                1: [
                    (1.0, 2, 0, False)
                ],  # from state 1, action 1 (right) leads to state 2 on the right with reward 0
            },
            2: {
                0: [
                    (1.0, 1, 0, False)
                ],  # from state 2, action 0 (left) leads to state 1 on the left with reward 0
                1: [
                    (1.0, 3, 1, True)
                ],  # from state 2, action 1 (right) leads to state 3 on the right with reward 1 and done = True. Goal state reached
            },
            3: {  # This is a terminal state.  Agent stays in the same position with reard 0
                0: [(1.0, 3, 0, True)],
                1: [(1.0, 3, 0, True)],
            },
        }
        
    def get_trasition(self, state, action):
        return self.P[state][action]
    
    
env = GridWorld()

In [3]:
# Sample state transition

env.get_trasition(2,0)

[(1.0, 1, 0, False)]

## Policy Evaluation

In [4]:
# Create a uniform random policy

def create_policy():
    policy = np.ones([env.n_states, env.n_actions]) / env.n_actions
    return policy

policy = create_policy()
policy

array([[0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5],
       [0.5, 0.5]])

In [5]:
def policy_evaluation(policy, env, gamma=0.3, theta=1e-6):
    V = np.zeros(env.n_states)  # Initialize value functions to zero
    while True:
        delta = 0  # Track convergence
        for s in range(env.n_states):
            v = 0
            for a, action_prob in enumerate(policy[s]):
                for prob, next_state, reward, done in env.get_trasition(s, a):
                    v += action_prob * (reward + gamma * prob * V[next_state])
            delta = max(delta, abs(V[s] - v))
            V[s] = v
            
        if delta < theta:  # Stop when value stabilizes
            break
    
    return V

In [6]:
V = policy_evaluation(policy, env, gamma=0.2, theta=1e6)

print("Value function from policy evaluation: ", V)

Value function from policy evaluation:  [0.  0.  0.5 0. ]


## Policy Improvement

In [7]:
def policy_improvement(V, env, gamma=0.9):
    policy = np.zeros([env.n_states, env.n_actions])
    
    for s in range(env.n_states):
        q_values = np.zeros(env.n_actions) # Action values
        for a in range(env.n_actions):
            for prob, next_state, reward, done in env.get_trasition(s, a):
                q_values[a] += prob * (reward + gamma * V[next_state])
        best_action = np.argmax(q_values)
        policy[s, best_action] = 1.0  # Assign probability 1 to the best action
    
    return policy

In [8]:
policy_improvement(V, env, gamma=0.9)

array([[1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.]])